In [0]:
from collections import Counter
import sys
import time
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model

In [58]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset (2).csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

In [0]:
df = df.drop(['Activity', 'Time', 'timestamp'], axis = 1) 
df = df[(df.T != 0).any()]
df = df[(df == 1).sum(1) < 2]

In [161]:
df_s = df
df.head(10)

,officeCarp,livingCarp,kitchenCarp,hallwayCarp,bedroomCarp,bathroomCarp
0,0,0,0,0,1,0
2,0,0,0,0,0,1
4,0,0,0,0,1,0
6,0,0,0,1,0,0
7,0,0,1,0,0,0
9,0,0,1,0,0,0
10,0,0,0,1,0,0
12,0,1,0,0,0,0
14,0,1,0,0,0,0
15,0,0,0,1,0,0


In [162]:
X_raw = df_s.values
X = []
rows = X_raw.shape[0]
cols = X_raw.shape[1]
for x in range(0, rows):
    for y in range(0, cols):
        if X_raw[x,y] == 1:
          X.append(y)
print(X)

[4, 5, 4, 3, 2, 2, 3, 1, 1, 3, 5, 3, 0, 0, 0, 3, 5, 3, 3, 5, 3, 4, 3, 2, 2, 3, 1, 3, 2, 3, 1, 5, 3, 1, 3, 0, 3, 5, 3, 1, 1, 3, 2, 3, 5, 3, 0, 3, 1, 3, 4, 5, 4, 5, 4, 4, 5, 4, 5, 3, 2, 3, 3, 0, 3, 5, 3, 0, 3, 3, 5, 3, 1, 3, 3, 1, 3, 3, 5, 3, 1, 5, 3, 3, 4, 5, 4, 5, 3, 2, 3, 3, 0, 3, 5, 3, 0, 3, 3, 2, 3, 3, 5, 3, 4, 4, 5, 3, 5, 3, 4, 3, 2, 3, 0, 3, 3, 2, 3, 3, 5, 3, 4, 4, 5, 3, 2, 3, 0, 0, 3, 3, 2, 3, 3, 5, 3, 4, 4, 5, 3, 2, 3, 0, 0, 3, 3, 2, 3, 3, 5, 3, 4, 4, 5, 3, 2, 3, 4, 3, 1, 3, 3, 5, 3, 3, 0, 3, 3, 5, 3, 2, 3, 4, 4, 5, 4, 3, 2, 2, 3, 1, 1, 3, 5, 3, 0, 0, 0, 3, 5, 3, 3, 5, 3, 0, 3, 1, 1, 3, 5, 3, 2, 3, 4, 5, 3, 4, 4, 5, 4, 3, 2, 3, 0, 3, 3, 2, 3, 3, 5, 3, 4, 4, 5, 4, 5, 3, 2, 3, 3, 0, 3, 5, 3, 0, 3, 3, 5, 3, 2, 2, 3, 1, 3, 5, 3, 1, 1, 3, 0, 0, 3, 3, 4, 4, 5, 3, 2, 3, 1, 4, 5, 4, 4, 5, 3, 4, 3, 2, 3, 0, 3, 3, 0, 3, 5, 3, 4, 3, 2, 3, 1, 3, 2, 3, 1, 3, 5, 3, 0, 3, 5, 3, 4, 4, 5, 3, 5, 3, 4, 3, 2, 3, 0, 3, 3, 5, 3, 3, 2, 3, 0, 3, 1, 1, 0, 5, 1, 0, 3, 4, 5, 3, 4, 3, 2, 3, 0, 3, 3, 0, 3, 

In [163]:
x0, x1, x2, x3, x4, x5 = [], [], [], [], [], []
Xpair = [x0, x1, x2, x3, x4, x5]
for i in range(len(X) - 1):
  if(X[i] == 0):
    Xpair[0].append(X[i+1])
  elif(X[i] == 1):
    Xpair[1].append(X[i+1])
  elif(X[i] == 2):
    Xpair[2].append(X[i+1])
  elif(X[i] == 3):
    Xpair[3].append(X[i+1])
  elif(X[i] == 4):
    Xpair[4].append(X[i+1])
  elif(X[i] == 5):
    Xpair[5].append(X[i+1])

print(Xpair)

[[0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 2, 3, 0, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 5, 3, 3, 0, 3, 5, 3], [1, 3, 3, 5, 3, 1, 3, 3, 3, 3, 5, 3, 1, 3, 1, 3, 3, 1, 3, 4, 3, 3, 1, 0, 0, 3, 3, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 1, 3, 5, 3, 3, 3, 1, 3, 3, 1, 3, 1, 3, 3, 3, 2, 1, 0, 5, 3, 3, 3, 3, 5, 3, 1, 3, 3, 3, 1, 3, 2, 3, 3, 1, 0, 5], [2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3], [2, 1, 5, 0, 5, 3, 5, 4, 2, 1, 2, 1, 1, 0, 5, 1, 2, 5, 0, 1, 4, 2, 3, 0, 5, 0, 3, 5, 1, 3, 1, 3, 5, 1, 3, 4, 2, 3, 0, 5, 0, 3, 2, 3, 5, 4, 5, 4, 2, 0, 3, 2, 3, 5, 4, 2, 0, 3, 2, 3, 5, 4, 2, 0, 3, 2, 3, 5, 4, 2, 4, 1, 3, 5, 3, 0, 3, 5, 2, 4, 2, 1, 5, 0, 5, 3, 5, 0, 1, 5, 2, 4, 4, 2, 0, 3, 2, 3,

In [164]:
x0, x1, x2, x3, x4, x5 = [], [], [], [], [], []
tX = [x0, x1, x2, x3, x4, x5]

for j in range(6):
  c0, c1, c2, c3, c4, c5 = 0,0,0,0,0,0
  for i in range(len(Xpair[j])):
    if(Xpair[j][i] == 0):
      c0 = c0 + 1
    elif(Xpair[j][i] == 1):
      c1 = c1 + 1
    elif(Xpair[j][i] == 2):
      c2 = c2 + 1
    elif(Xpair[j][i] == 3):
      c3 = c3 + 1
    elif(Xpair[j][i] == 4):
      c4 = c4 + 1
    elif(Xpair[j][i] == 5):
      c5 = c5 + 1

  cnt = [c0, c1, c2, c3, c4, c5]
  tot = len(Xpair[j])
  
  for i in range(6):
    if(tot > 0):
      tX[j].append(round(cnt[i]/tot, 2))
  print(tX[j])

[0.16, 0.01, 0.02, 0.77, 0.0, 0.04]
[0.05, 0.22, 0.03, 0.61, 0.01, 0.08]
[0.0, 0.0, 0.08, 0.92, 0.0, 0.0]
[0.16, 0.14, 0.16, 0.18, 0.15, 0.2]
[0.0, 0.0, 0.0, 0.3, 0.24, 0.46]
[0.01, 0.01, 0.0, 0.85, 0.12, 0.0]


In [165]:
fin_t = [
[0.16, 0.01, 0.02, 0.77, 0.0, 0.04],
[0.05, 0.22, 0.03, 0.61, 0.01, 0.08],
[0.0, 0.0, 0.08, 0.92, 0.0, 0.0],
[0.16, 0.14, 0.16, 0.19, 0.15, 0.2],
[0.0, 0.0, 0.0, 0.3, 0.24, 0.46],
[0.01, 0.01, 0.0, 0.86, 0.12, 0.0],
]

print(fin_t)

[[0.16, 0.01, 0.02, 0.77, 0.0, 0.04], [0.05, 0.22, 0.03, 0.61, 0.01, 0.08], [0.0, 0.0, 0.08, 0.92, 0.0, 0.0], [0.16, 0.14, 0.16, 0.19, 0.15, 0.2], [0.0, 0.0, 0.0, 0.3, 0.24, 0.46], [0.01, 0.01, 0.0, 0.86, 0.12, 0.0]]


In [171]:
npa = np.asarray(fin_t, dtype=np.float32)
two_step = npa.dot(npa)
two_step = np.round(two_step, 4)
print(two_step)

[[0.1497 0.112  0.1283 0.3284 0.1204 0.1612]
 [0.1174 0.1351 0.1076 0.388  0.1057 0.1462]
 [0.1472 0.1288 0.1536 0.2484 0.138  0.184 ]
 [0.065  0.061  0.0506 0.6089 0.0899 0.1246]
 [0.0526 0.0466 0.048  0.5246 0.1578 0.1704]
 [0.1397 0.1227 0.1381 0.2132 0.1579 0.2284]]
